In [126]:
from qibullet import SimulationManager
from qibullet import NaoVirtual , NaoFsr
import matplotlib.pyplot as plt
import numpy as np
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import pickle

In [127]:
if torch.cuda.is_available():
  device = "cuda" 
else:
  device = "cpu"
print(device, " in use")

cpu  in use


In [131]:
obs_space = 10

class ActorNet(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
      super(ActorNet, self).__init__()
      self.rnn = nn.RNN(input_dim, hidden_dim)
      #self.fc = nn.Linear(hidden_dim, output_dim)
      self.fc1 = nn.Linear(hidden_dim, hidden_dim)
      self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
      out, _ = self.rnn(x)
      #out = out[:, -1, :]
      #out = self.fc(out)
      out = self.fc1(out)
      out = self.fc2(out)
      return out
    
Actor = ActorNet(obs_space, 10, obs_space)

# load model weights
Actor.load_state_dict(torch.load('model_weights_day3_1.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [132]:
# Launch Simulation Environment

vis = True
simulation_manager = SimulationManager()
nao_sim = simulation_manager.launchSimulation(gui=vis, auto_step=True)
simulation_manager.setGravity(nao_sim, [0.0, 0.0, -9.81])
nao = None


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=VMware, Inc.
GL_RENDERER=llvmpipe (LLVM 10.0.0, 256 bits)
GL_VERSION=3.3 (Core Profile) Mesa 20.0.8
GL_SHADING_LANGUAGE_VERSION=3.30
pthread_getconcurrency()=0
Version = 3.3 (Core Profile) Mesa 20.0.8
Vendor = VMware, Inc.
Renderer = llvmpipe (LLVM 10.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = VMware, Inc.
ven = VMware, Inc.


In [133]:
# Utility Functions
# Joints to monitor
legJoints = [ "LHipRoll", "LHipPitch", "LKneePitch", "LAnklePitch", "LAnkleRoll",
              "RHipRoll", "RHipPitch", "RKneePitch", "RAnklePitch", "RAnkleRoll"]

min_values = torch.tensor([-0.37, -1.53, -0.09, -1.18, -0.39 , -0.79, -1.53, -0.10, -1.18, -0.76], dtype=torch.float32)
max_values = torch.tensor([ 0.79, 0.48, 2.11, 0.92, 0.76,  0.37, 0.48, 2.12, 0.93, 0.39], dtype=torch.float32)

nao_prev_position_X = 0
nao_prev_position_Y = 0

prev_weight_cntr = 0
def env_Reward():
    global prev_weight_cntr
    x, y, z = nao.getPosition()
    terminated = False
    delX = nao_prev_position_X - x 
    delY = nao_prev_position_Y - x 
    weight = -(nao.getTotalFsrValues(NaoFsr.LFOOT) + nao.getTotalFsrValues(NaoFsr.RFOOT) )

    # if robot fallen down
    if weight == 0: 
        reward = -1
        prev_weight_cntr += 1
    else:
        reward = delX
        prev_weight_cntr = 0

    if prev_weight_cntr == 5:
        terminated = True
        prev_weight_cntr = 0
    return reward, terminated

def env_state():
    return nao.getAnglesPosition(legJoints)
    
def env_action(angles, speed):
    nao.setAngles(legJoints, angles ,speed)

def env_itrm_step(itr):
    for _ in range(itr):
        simulation_manager.stepSimulation(nao_sim)

def env_reset():
    global nao_prev_position_X , nao_prev_position_Y
    global nao
    simulation_manager.resetSimulation(nao_sim)
    simulation_manager.setGravity(nao_sim, [0.0, 0.0, -9.81])
    nao = simulation_manager.spawnNao( 
    nao_sim,
    translation=[0, 0, 0],quaternion=[0, 0, 0, 1],
    spawn_ground_plane=True)
    env_itrm_step(100)
    nao_prev_position_X, nao_prev_position_Y, _ = nao.getPosition()


def env_stop():
    simulation_manager.stopSimulation(nao_sim)
    # wait for limited iterations 

def env_step(angles):
    env_action(angles, 0.1)
    #env_itrm_step(50)
    reward, terminated = env_Reward()
    next_state = env_state()
    return next_state, reward, terminated

env_reset()
    

In [134]:
for _ in range(10):
    state =  nao.getAnglesPosition(legJoints)
    total_reward = 0
    itr = 0
    while True:
        itr += 1
        x = torch.tensor(state).float()
        x = x.view(1,10)
        next_state = Actor(x)
        next_state = torch.clamp(next_state[0], min_values, max_values)   
        angles = next_state.tolist()
        #state[0] = (itr%3)/10  
        next_state, reward, terminated = env_step(angles)
        total_reward += reward
        state = next_state
        
        time.sleep(0.5)
        if terminated:
            env_reset()
            break

In [119]:
env_reset()

In [130]:
env_stop()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [ ]:
1 +1

In [ ]:
torch.__version__ 

'2.0.1'

In [ ]:
print(torch.cuda.is_available())

In [41]:
events = nao.get_data()

AttributeError: 'NaoVirtual' object has no attribute 'get_data'

In [62]:
values = nao.getTotalFsrValues(NaoFsr.LFOOT)
values

0.0

In [93]:
nao.subscribeImu(frequency=10)
acc_values , akak = nao.getImuValues()
acc_values , akak

((9.662449090932986e-06, 4.319630726013825e-05, -3.8398855511636495e-05),
 [5.382426428431004e-10, -2.470350037125424e-10, -1.6765625098147067e-10])

In [64]:
value = nao.getFsrValue(NaoFsr.LFOOT_FL)
value

-0.0